In [1]:
import pandas as pd
from hardware.opentrons.opentrons_api import OpentronsAPI
from hardware.cameras import PendantDropCamera
from hardware.sensor.sensor_api import SensorAPI 
from hardware.opentrons.formulater import Formulater
from hardware.opentrons.configuration import Configuration
from analysis.plots import Plotter
from utils.logger import Logger
from utils.load_save_functions import load_settings
from utils.search_containers import get_well_id_concentration, get_well_id_solution
from analysis.active_learning import ActiveLearner
from analysis.models import szyszkowski_model
from hardware.opentrons.droplet_manager import DropletManager
from utils.load_save_functions import save_calibration_data


opentrons_api = OpentronsAPI()
opentrons_api.initialise()
pd_cam = PendantDropCamera()
sensor_api = SensorAPI()
config = Configuration(opentrons_api=opentrons_api)
labware = config.load_labware()
containers = config.load_containers()
pipettes = config.load_pipettes()
left_pipette = pipettes["left"]
right_pipette = pipettes["right"]
plotter = Plotter()
settings = load_settings()
logger = Logger(
    name="protocol",
    file_path=f'experiments/{settings["EXPERIMENT_NAME"]}/meta_data',
)

droplet_manager = DropletManager(
    left_pipette=left_pipette,
    containers=containers,
    pendant_drop_camera=pd_cam,
    opentrons_api=opentrons_api,
    plotter=plotter
)

formulater = Formulater(
    left_pipette=left_pipette,
    right_pipette=right_pipette,
    containers = containers,
)

C:\Users\BigChemistry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-01 13:53:21,533 - INFO - Protocol already uploaded, using existing protocol.
2025-04-01 13:53:31,930 - INFO - Run created succesfully (ID: d57b1c0b-efa6-4658-bea2-7207ca1a9958).
2025-04-01 13:53:34,796 - INFO - All custom labware definitions added.
2025-04-01 13:53:36,463 - INFO - Labware loaded successfully.
2025-04-01 13:53:36,468 - INFO - Containers loaded successfully.
2025-04-01 13:53:40,598 - INFO - Pipettes loaded successfully.


In [2]:
opentrons_api.home()

2025-04-01 13:54:06,886 - INFO - Robot homed.


In [9]:
from utils.search_containers import get_well_id_solution

In [10]:
right_pipette.drop_tip()
# right_pipette.transfer(20, source=containers["7A1"], destination=containers["7A1"])

2025-04-01 14:19:27,379 - INFO - Right pipette dropped tip into trash.


In [ ]:
offset = {"x": -1, "y": 1.4, "z": -10}
right_pipette.move_to_well_calibrate(
    container=containers["7A1"], 
    well="A1", 
    offset=offset
)

In [6]:
get_well_id_solution(containers=containers, solution_name="trash")

'5B1'

In [11]:
rows = ["7A","7B","7C","7D","7E"]

formulater.symm_seeding_dilution(
    row_id=rows,
    column=1,
    total_well_volume=300,
    solution_name1="CTAB",
    solution_name2="DDAB"
)

formulater.seeded_serial_dilution(
    row_id=rows,
    n_dilutions=8,
    well_volume=150
)
# right_pipette.drop_tip()

2025-04-01 14:19:35,794 - INFO - Picked up tip.
2025-04-01 14:19:35,794 - INFO - Start of seeding in row(s) 7A, 7B, 7C, 7D, 7E, with varying amounts of CTAB and DDAB.
2025-04-01 14:19:35,794 - INFO - Transferring 270.0 uL from 5A1 to well 7A1 with right pipette.
2025-04-01 14:20:48,785 - INFO - Done with mixing in 5A1 with order after, with volume 250.0 uL, repeated 12 times
2025-04-01 14:20:55,449 - INFO - Touched tip performed, repeated 1 times
2025-04-01 14:20:55,449 - INFO - Container: aspirated 270.0 uL from this container with content 1.69 mM CTAB.
2025-04-01 14:20:55,449 - INFO - Aspirated 270.0 uL from 5A1 with right pipette.
2025-04-01 14:22:10,340 - INFO - Done with mixing in 7A1 with order after, with volume 250.0 uL, repeated 12 times
2025-04-01 14:22:14,740 - INFO - Touched tip performed, repeated 1 times
2025-04-01 14:22:14,741 - INFO - Container: dispensed 270.0 uL into this container from source 5A1 containing 1.69 mM CTAB.
2025-04-01 14:22:14,742 - INFO - Dispensed 270

KeyError: '7B1'

In [ ]:
rows = ["7A","7B","7C","7D","7E"]

right_pipette.pick_up_tip()
for row in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(270-(60*row), source=containers["5A2"], destination=containers[f"{rows[row]}1"], mix=("both", 110, 3), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

flipped_rows = list(reversed(rows))

right_pipette.pick_up_tip()
for row in range(len(flipped_rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(270-(60*row), source=containers["5A3"], destination=containers[f"{flipped_rows[row]}1"], mix=("both", 110, 3), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

formulater.seeded_serial_dilution(
    row_id=rows,
    n_dilutions=8,
    well_volume=150
)

In [ ]:
## Measuring script

right_pipette.pick_up_tip()
for row in ["A","D","G"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 2):               # By default blowout occurs before touch tip
        well = f"7{row}{col}"
        right_pipette.transfer(150, source=containers["5A1"], destination=containers[well], touch_tip = True, blow_out=True)
right_pipette.transfer(150, source=containers["5A1"], destination=containers["7H1"], touch_tip = True, blow_out=True)   # Blank measurement
right_pipette.drop_tip()        # Takes 14.25 min for this part for 5 rows, with new touch_tip

rows = ["D"]
right_pipette.pick_up_tip()         #5A3
for i in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(75, source=containers["5A1"], destination=containers[f"7{rows[i]}1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.transfer(75, source=containers["5A1"], destination=containers[f"7G1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.transfer(75, source=containers["5A1"], destination=containers[f"7A1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

rows = ["D"]
right_pipette.pick_up_tip()
for i in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(75, source=containers["5A1"], destination=containers[f"7{rows[i]}1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.transfer(75, source=containers["5A1"], destination=containers[f"7A1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.transfer(75, source=containers["5A1"], destination=containers[f"7G1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

rows = ["A", "D", "G"]
for row in range(len(rows)):
    right_pipette.pick_up_tip()
    for i in range(1, 2):
        right_pipette.transfer(150, source=containers[f"7{rows[row]}{i}"], destination=containers[f"7{rows[row]}{i+1}"], mix=("both", 50, 9), blow_out=True, touch_tip=True)
    right_pipette.drop_tip()        # approx 13 min 

# for row in range(len(rows)):
#     right_pipette.pick_up_tip()
#     right_pipette.transfer(150, source=containers[f"10{rows[row]}8"], destination=containers["5B1"], mix=("before", 100, 9), blow_out=True)
    # right_pipette.drop_tip()

2025-03-21 09:34:44,472 - INFO - Picked up tip.
2025-03-21 09:34:44,489 - INFO - Transferring 150 uL from 5A1 to well 7A1 with right pipette.
2025-03-21 09:34:51,686 - INFO - Touched tip performed, repeated 1 times
2025-03-21 09:34:51,686 - INFO - Container: aspirated 150 uL from this container with content pure mM water.
2025-03-21 09:34:51,686 - INFO - Aspirated 150 uL from 5A1 with right pipette.
2025-03-21 09:34:58,417 - INFO - blow out done in container 7A1
2025-03-21 09:35:00,600 - INFO - Touched tip performed, repeated 1 times
2025-03-21 09:35:00,600 - INFO - Container: dispensed 150 uL into this container from source 5A1 containing pure mM water.
2025-03-21 09:35:00,600 - INFO - Dispensed 150 uL into well 7A1 with right pipette.
2025-03-21 09:35:00,600 - INFO - Transferring 150 uL from 5A1 to well 7D1 with right pipette.
2025-03-21 09:35:09,631 - INFO - Touched tip performed, repeated 1 times
2025-03-21 09:35:09,631 - INFO - Container: aspirated 150 uL from this container with 

KeyboardInterrupt: 

In [ ]:
## Measuring script

right_pipette.pick_up_tip()
for row in ["A", "B", "C", "D", "E", "F", "G"]:     # Loop to fill with 150 ul of water in all used wells
    for col in range(1, 9):               # By default blowout occurs before touch tip
        well = f"10{row}{col}"
        right_pipette.transfer(150, source=containers["5A1"], destination=containers[well], touch_tip = True, blow_out=True)
right_pipette.transfer(150, source=containers["5A1"], destination=containers["10H1"], touch_tip = True, blow_out=True)   # Blank measurement
right_pipette.drop_tip()        # Takes 14.25 min for this part for 5 rows, with new touch_tip

rows = ["F", "E", "D", "C", "B"]
right_pipette.pick_up_tip()
for i in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(135-(30*i), source=containers["5A3"], destination=containers[f"10{rows[i]}1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.transfer(145, source=containers["5A3"], destination=containers[f"10G1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.transfer(5, source=containers["5A3"], destination=containers[f"10A1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

rows = ["B", "C", "D", "E", "F"]
right_pipette.pick_up_tip()
for i in range(len(rows)):  # Loop through rows, but stop before the last one
    right_pipette.transfer(135-(30*i), source=containers["5A2"], destination=containers[f"10{rows[i]}1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.transfer(145, source=containers["5A3"], destination=containers[f"10A1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.transfer(5, source=containers["5A3"], destination=containers[f"10G1"], mix=("both", 90, 12), blow_out=True, touch_tip=True)
right_pipette.drop_tip()

rows = ["A", "B", "C", "D", "E", "F", "G"]
for row in range(len(rows)):
    right_pipette.pick_up_tip()
    for i in range(1, 9):
        right_pipette.transfer(150, source=containers[f"10{rows[row]}{i}"], destination=containers[f"10{rows[row]}{i+1}"], mix=("both", 50, 9), blow_out=True, touch_tip=True)
    right_pipette.drop_tip()        # approx 13 min 

# for row in range(len(rows)):
#     right_pipette.pick_up_tip()
#     right_pipette.transfer(150, source=containers[f"10{rows[row]}8"], destination=containers["5B1"], mix=("before", 100, 9), blow_out=True)
    # right_pipette.drop_tip()